# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#498 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846851422636                   -0.70    4.0
  2   -7.852316542277       -2.26       -1.53    1.0
  3   -7.852646082780       -3.48       -2.53    3.2
  4   -7.852646678462       -6.22       -3.35    2.5
  5   -7.852646686024       -8.12       -4.82    1.0
  6   -7.852646686726       -9.15       -5.25    4.0
  7   -7.852646686730      -11.46       -6.00    2.0
  8   -7.852646686730      -12.69       -7.42    2.0
  9   -7.852646686730   +  -14.75       -7.87    2.2
 10   -7.852646686730   +    -Inf       -9.29    1.5
 11   -7.852646686730      -15.05       -9.60    2.0
 12   -7.852646686730   +  -15.05       -9.27    1.0
 13   -7.852646686730   +    -Inf       -9.99    1.0
 14   -7.852646686730   +    -Inf       -9.33    1.0
 15   -7.852646686730   +    -Inf       -9.59    1.0
 16   -7.852646686730   +  -15.05      -10.09    1.0
 17   -7.852646686730      -14.75      -10.10 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846898743548                   -0.70           4.2
  2   -7.852526382556       -2.25       -1.62   0.80    5.0
  3   -7.852609642546       -4.08       -2.79   0.80    5.2
  4   -7.852646567753       -4.43       -3.57   0.80    7.2
  5   -7.852646683623       -6.94       -4.54   0.80    8.8
  6   -7.852646686682       -8.51       -5.13   0.80   10.2
  7   -7.852646686725      -10.36       -6.22   0.80   11.2
  8   -7.852646686730      -11.31       -7.17   0.80   13.5
  9   -7.852646686730      -13.67       -7.76   0.80   15.2
 10   -7.852646686730   +    -Inf       -8.75   0.80   16.5
 11   -7.852646686730      -15.05       -9.56   0.80   18.2
 12   -7.852646686730   +    -Inf      -10.44   0.80   19.0
 13   -7.852646686730      -15.05      -11.39   0.80   21.2
 14   -7.852646686730   +    -Inf      -11.90   0.80   23.0
 15   -7.852646686730   +  -14.57      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.406275e+01     3.251672e+00
 * time: 0.4294090270996094
     1     1.067972e+00     2.100281e+00
 * time: 0.6480839252471924
     2    -1.531814e+00     1.914626e+00
 * time: 0.6695280075073242
     3    -3.671105e+00     1.898101e+00
 * time: 0.7002639770507812
     4    -4.903728e+00     1.570257e+00
 * time: 0.730720043182373
     5    -6.596413e+00     9.301106e-01
 * time: 0.7611861228942871
     6    -7.321298e+00     4.800931e-01
 * time: 0.79180908203125
     7    -7.617320e+00     3.399411e-01
 * time: 0.8131890296936035
     8    -7.713846e+00     1.072469e-01
 * time: 0.8344769477844238
     9    -7.772163e+00     1.651943e-01
 * time: 0.8557651042938232
    10    -7.802223e+00     6.854289e-02
 * time: 0.877249002456665
    11    -7.826705e+00     6.247492e-02
 * time: 0.8990259170532227
    12    -7.843572e+00     5.749179e-02
 * time: 0.9205701351165771
    13    -7.849674e+00     2.894329e-02
 * time: 0.94224691390991

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846863043185                   -0.70    4.2
  2   -7.852328536967       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686708                   -2.54
  2   -7.852646686730      -10.67       -5.89
  3   -7.852646686730   +  -14.75      -12.56
|ρ_newton - ρ_scf|  = 1.7873630398571375e-13
|ρ_newton - ρ_scfv| = 8.863321518121582e-14
|ρ_newton - ρ_dm|   = 3.739244240431845e-9
